Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Assignment)

This notebook is for you to practice skills during lecture.

Today's guided module project and assignment will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills.

## Sections
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy
* <a href="#p4">Part 4</a>: Post Lecture Assignment

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

### Load Competition Data

In [75]:
import pandas as pd

# You may need to change the path
train = pd.read_csv('./whiskey-reviews-dspt4/train.csv')
test = pd.read_csv('./whiskey-reviews-dspt4/test.csv')
print(train.shape, test.shape)

(4087, 3) (1022, 2)


In [2]:
train.head()

,id,description,ratingCategory
0,1321,"\nSometimes, when whisky is batched, a few lef...",1
1,3861,\nAn uncommon exclusive bottling of a 6 year o...,0
2,655,\nThis release is a port version of Amrut’s In...,1
3,555,\nThis 41 year old single cask was aged in a s...,1
4,1965,"\nQuite herbal on the nose, with aromas of dri...",1


In [76]:
# Distribution of ratingCategory: 0 (Excellent), 1 (Good), 2 (Poor)
train.ratingCategory.value_counts(normalize=True)

1    0.704918
0    0.279178
2    0.015904
Name: ratingCategory, dtype: float64

In [77]:
# Read a few reviews from the "Excellent" category
pd.set_option('display.max_colwidth', 0)
train[train.ratingCategory == 0].sample(3)

,id,description,ratingCategory
2973,4004,"\nDrawn from a single refill barrel, the nose is like a day in the life of a honeybee: streams of pollen, rich nectar, heady scents of flying over bright summer flowers and summer picnics of white peaches and lemonade before returning to the hive to the sweet, sticky honey. It’s quite a buzz! There’s an acidic palate of lemon juice, lime, honey, and lemon pith bordering on bitterness. The finish is incredibly dry, like someone just invented lemon-scented sandpaper. (168 bottles) £85",0
1946,5002,"\nThis literally tastes like pan-fried mushrooms and roots. It’s earthy; you don’t really expect such intensity from a flavored whiskey. After earth, hops, chocolate, and root beer, I finally taste the whiskey; a nearly absent sensation toward the end. But I guess that’s the point: if they wanted you to taste whiskey, they wouldn’t flavor it. Still, there are no chemically-imbalanced flavors here. This is worth trying, even for us curmudgeon purists.",0
156,4425,"\nHighwood Distillers has returned from their devastating 2013 flood with better whisky than ever. A soft, sweet, fruity nose understates the power of an intensely spicy palate chock-full of sour, bitter kumquats. Subtle honey flavors abound, with delicate threads of golden burley tobacco entwined. It’s a simple dram for mixing, but with plenty of interest to satisfy those who would rather sip. Add ginger ale and ice, or drink straight up. (Canada only) C$30",0


In [5]:
# Read a few reviews from the "Poor" category
train[train.ratingCategory == 2].sample(3)

,id,description,ratingCategory
1849,5037,"\nThis is new make from the Rittenhouse Rye mashbill, unaged, right off the still. Yeasty mash notes come through strongly, with a big nose of rye and corn underneath. It smells fresh, and alive. Wow, that’s powerful stuff. Much hotter than the Trybox Corn, this one is on fire with rye spice; it’s got me breaking a sweat! A bit of water brings out more grain notes, and some tempering sweetness, but it’s still no pussycat. Cleanly powerful.",2
2161,5046,"\nThis may be the best-known Belgian single grain whisky and it vies for attention with its perfumed nose of violet creams, acetone, and peach juice with ripe mango and papaya. Unfortunately, the body is lacking, yielding up a juicy collision of tropical fruits and icing sugar sweetness, but after that initial arrival, the torpid profile is rather unremarkable. The flavors die quickly, with some residual flavor trails persisting. All in all, a bit drab. €53",2
2193,5064,"\nWhen they say berry, they mean berries: baskets of blueberries, blackberries, and raspberries, until suddenly the palate veers off into grape popsicles. This is one sweet, unabashedly faux-fruity potation, but other than traces of oak, not much whisky flavor remains. Still, in the right hands it could be a lot of cocktail fun. Rather than serving it in a whisky glass, a dram or two on French vanilla or coconut-milk ice cream is genuinely scrumptious.",2


### Split the Training Set into Train/Validation

In [78]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train['description'], 
                                                    train['ratingCategory'], 
                                                    test_size=0.2, 
                                                    stratify=train['ratingCategory'],
                                                    random_state=42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
# u 

# Cross Validation Set - could be used to assert your training set

(3269,) (818,) (3269,) (818,)


### Define Pipeline Components

In [79]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

In [9]:
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
rfc = RandomForestClassifier()

dtm = vect.fit_transform(train)

pipe = Pipeline([('vect', vect), ('clf', rfc)])
# clf - classifier

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [10]:
parameters = {
    'vect__max_df': (0.4, 0.7),
    'vect__min_df': (2, 5, 10),
    'clf__n_estimators': (100, 500),
    'clf__max_depth': (10, 20, None)}

grid_search = GridSearchCV(pipe, parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   13.8s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  1.8min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 2),
                                                        no

In [11]:
grid_search.best_score_

0.7421273726063194

In [12]:
grid_search.best_params_

{'clf__max_depth': None,
 'clf__n_estimators': 500,
 'vect__max_df': 0.4,
 'vect__min_df': 5}

### Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so you only submit if you feel you cannot achieve higher test accuracy. For this competition the max daily submissions are capped at **20**. Submit for each demo and for your assignment. 

In [13]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [14]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [15]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [16]:
subNumber = 0

In [58]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./whiskey-reviews-dspt4/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

You're trying to achieve a minimum of 70% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


### Define Pipeline Components

In [80]:
import scipy.stats as stats
from sklearn.model_selection import RandomizedSearchCV
from sklearn.decomposition import TruncatedSVD

In [81]:
lsi = TruncatedSVD(algorithm='randomized')
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2),
                      min_df=5,
                      max_df=0.4)
clf = RandomForestClassifier(n_estimators=100,
                            max_depth=None)

pipe = Pipeline([('vect', vect), # Note to self: order of pipeline is important
                 ('lsi', lsi),
                 ('clf', clf)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [20]:
parameters = {
    'lsi__n_components': [250,1000],
    'lsi__n_iter': [20,100]
}

grid_search = GridSearchCV(pipe, parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  3.3min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=0.4,
                                                        max_features=None,
                                                        min_df=5,
                                                        ngram_range=(1, 2),
                                                        no

In [21]:
grid_search.best_score_

0.7124497145613518

In [ ]:
grid_search.best_params_

### Make a Submission File

In [ ]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [ ]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [ ]:
# Make Sure the Category is an Integer
submission.head()

In [41]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./whiskey-reviews-dspt4/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

## Follow Along

In [82]:
# Apply to your Dataset
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from scipy.stats import randint

param_dist = {
    'max_depth' : randint(3,10),
    'min_samples_leaf': randint(2,15)
}

In [26]:
import spacy
nlp = spacy.load('en_core_web_lg')

In [83]:
clf = RandomForestClassifier(n_estimators=100,
                            max_depth=None)

In [84]:
def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [85]:
X_train = get_word_vectors(X_train)
len(X) == len(X_train)

True

In [86]:
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [87]:
yt = clf.predict(X_train)

In [89]:
from sklearn.metrics import accuracy_score

In [93]:
accuracy_score(y_train, yt)

1.0

In [94]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(solver='lbfgs',
                  alpha =1e-5,
                  hidden_layer_sizes=(16,2),
                  random_state=42)

In [95]:
clf.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(16, 2), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=42, shuffle=True, solver='lbfgs',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [96]:
X_test = get_word_vectors(X_test)

In [97]:
y_test = clf.predict(X_test)

In [100]:
accuracy_score(X_test, y_test)

ValueError: Classification metrics can't handle a mix of continuous-multioutput and binary targets

### Make a Submission File

In [ ]:
# Predictions on test sample
pred = ...predict(test['description'])

In [ ]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [ ]:
# Make Sure the Category is an Integer
submission.head()

In [ ]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model
submission.to_csv(f'./whiskey-reviews-dspt4/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# Post Lecture Assignment
<a id="p4"></a>

Your primary assignment this afternoon is to achieve a minimum of 70% accuracy on the Kaggle competition. Once you have achieved 70% accuracy, please work on the following: 

1. Research "Sentiment Analysis". Provide answers in markdown to the following questions: 
    - What is "Sentiment Analysis"? 
    - Is Document Classification different than "Sentiment Analysis"? Provide evidence for your response
    - How do create labeled sentiment data? Are those labels really sentiment?
    - What are common applications of sentiment analysis?
2. Research our why word embeddings worked better for the lecture notebook than on the whiskey competition.
    - This [text classification documentation](https://developers.google.com/machine-learning/guides/text-classification/step-2-5) from Google might be of interest
    - Neural Networks are becoming more popular for document classification. Why is that the case?

# What is Sentiment Analysis?
The use of natural language processing, text analysis, computational linguistics, and biometics to systematically identify, extract, quantify, and study affective states and subjective info.
# Is Document Classification different than "Sentiment Analysis"? Provide evidence for your response
Document classification is used in the boarder scope of sentiment analysis where it's trying to study the subjectiveness of infomation using document classification. Ex: I'd really truly love going out in this weather! (Trying to determine whether or not this sentence is sarcastic)
# How do we create labeled sentiment data? Are those labels really sentiment? (sentiment data ex: a star review on a amazon review)
Labeled sentiment data is created manually when you and someone label the data itself. Those labels are really sentiment; it's the reason why they are manually labeled by people in the first place.
# What are common applications of sentiment analysis?
The common applications of sentiment analysis are classifying polarity of a given text,quantifying patterns in text, and detecting polarity of product and movie reivews.
# Research why word embeddings worked better for the lecture notebook than on the whiskey competition
